In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

In [12]:
# URL de base avec le paramètre page_number
base_url = "https://footballdatabase.com/league-scores/england-premier-league-2023-2024/{}"

# Nombre de pages à scraper
num_pages = 6  # Par exemple, scraper les 5 premières pages

# Liste pour stocker les résultats
results = []

# Boucle à travers les pages
for page_number in range(1, num_pages + 1):
    # URL de la page spécifique
    url = base_url.format(page_number)

    # Faire la requête GET
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Parser le contenu HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouver tous les matchs
        matches = soup.find_all('div', class_='club-gamelist-match')

        # Parcourir chaque match trouvé
        for match in matches:
            # Récupérer le nom de l'équipe à domicile
            home_team = match.find('a', class_='limittext').get('title')

            # Récupérer le nom de l'équipe à l'extérieur
            away_team = match.find('a', class_='sm_logo-name_away').get('title')

            # Récupérer le score
            score = match.find('div', class_='club-gamelist-match-score').text.strip()

            # Ajouter le match à la liste des résultats
            results.append([home_team, away_team, score])

    else:
        print(f"Erreur {response.status_code} lors de la récupération de la page {url}")

# Nom du fichier CSV à créer
csv_filename = 'matches.csv'

# Écriture des résultats dans un fichier CSV
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['HomeTeam', 'AwayTeam', 'Score'])  # Écrire l'en-tête des colonnes
    writer.writerows(results)

print(f"Données sauvegardées dans {csv_filename}")

Données sauvegardées dans matches.csv


In [2]:
# Charger le fichier CSV dans un DataFrame
file_path = 'matches.csv'
df = pd.read_csv(file_path)

# Dictionnaire de correspondance entre les noms actuels et les nouveaux noms
team_mapping = {
    "Arsenal": "Arsenal",
    "Chelsea FC": "Chelsea",
    "Liverpool FC": "Liverpool",
    "Luton Town F.C.": "Luton Town",
    "Burnley": "Burnley",
    "Brentford FC": "Brentford",
    "Brighton & Hove Albion": "Brighton",
    "Sheffield United FC": "Sheffield United",
    "Crystal Palace": "Crystal Palace",
    "Manchester City": "Man City",
    "Everton FC": "Everton",
    "AFC Bournemouth": "Bournemouth",
    "Newcastle United": "Newcastle",
    "West Ham United": "West Ham",
    "Fulham": "Fulham",
    "Wolverhampton Wanderers": "Wolves",
    "Aston Villa": "Aston Villa",
    "Nottingham Forest": "Nottm Forrest",
    "Tottenham Hotspur": "Tottenham",
    "Manchester United": "Man United"
}

# Remplacer les noms des équipes dans les colonnes HomeTeam et AwayTeam
df['HomeTeam'] = df['HomeTeam'].map(team_mapping)
df['AwayTeam'] = df['AwayTeam'].map(team_mapping)

# Fonction pour déterminer le résultat (1 pour victoire à domicile, 0 sinon)
def determine_result(row):
    home_score = int(row['Score'].split(' - ')[0])
    away_score = int(row['Score'].split(' - ')[1])
    if home_score > away_score:
        return 1
    else:
        return 0

# Appliquer la fonction pour créer la colonne Result
df['Result'] = df.apply(determine_result, axis=1)

# Sauvegarder le DataFrame modifié dans un nouveau fichier CSV
output_file_path = 'matches_transformed.csv'
df.to_csv(output_file_path, index=False)

print(f"Transformation terminée. Fichier sauvegardé sous : {output_file_path}")

Transformation terminée. Fichier sauvegardé sous : matches_transformed.csv
